# Indexation de table

## Pourquoi indexer une table ?

Jusqu'ici, on a vu que __pour extraire de l'information d'une table, il fallait toujours parcourir l'ensemble de cette table__. On peut aisément comprendre que ce coût est très important lorsqu'il s'agit de traiter des tables de grande taille.

Il devient alors intéressant de créer de nouvelles structures pour accélérer des recherches type. Comme on l'a vu (et on le détaillera un peu plus tard), les dictionnaires ont des qualités intrinsèques qui permettent de trouver une valeur efficacement, par clé plutôt que par indice. 

Il suffit donc de créer un nouveau dictionnaire, en choisissant les bonnes clés. Celles qui seront à l'origine des recherches futures.

Ce principe s'appelle l'__indexation__ : on crée un __dictionnaire, avec des clés judicieusement choisies, pour améliorer les performances de recherche dans une table__.

Dans le cas d'une table, __l'indexation consite donc souvent à créer un dictionnaire de dictionnaires__.

Reprenons notre exemple de table de naissances...

In [26]:
import csv

with open("244400404_prenoms-enfants-nes-nantes.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    table_naissances = [{key : value for key, value in element.items()} for element in reader]

print(table_naissances)

[{'Prénom': 'IRIS', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '25.0'}, {'Prénom': 'ROMY', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '19.0'}, {'Prénom': 'MARGOT', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '13.0'}, {'Prénom': 'AMIRA', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '11.0'}, {'Prénom': 'EMMY', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '11.0'}, {'Prénom': 'GARANCE', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '11.0'}, {'Prénom': 'LILOU', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '11.0'}, {'Prénom': 'ALICIA', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '10.0'}, {'Prénom': 'LISON', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '10.0'}, {'Prénom': 'COLINE', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '9.0'}, {'Prénom': 'LUCIE', 'Sexe': 'FILLE', 'Année de naissance': '2019', 'Occurence': '8.0'}, {'Prénom': 'ALBA', '

Pour la suite, on va __nettoyer cette table en convertissant en entiers les nombres__ qu'elle contient (années et occurrences de naissances) et en corrigeant la faute d'orthographe:

In [31]:
for bebe in table_naissances:
    bebe['Année de naissance'] = int(bebe['Année de naissance'])
    bebe['Occurrence'] = int(float(bebe['Occurence']))
    del(bebe['Occurence'])
    
print(table_naissances)

[{'Prénom': 'IRIS', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 25}, {'Prénom': 'ROMY', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 19}, {'Prénom': 'MARGOT', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 13}, {'Prénom': 'AMIRA', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 11}, {'Prénom': 'EMMY', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 11}, {'Prénom': 'GARANCE', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 11}, {'Prénom': 'LILOU', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 11}, {'Prénom': 'ALICIA', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 10}, {'Prénom': 'LISON', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 10}, {'Prénom': 'COLINE', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 9}, {'Prénom': 'LUCIE', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence': 8}, {'Prénom': 'ALBA', 'Sexe': 'FILLE', 'Année de naissance': 2019, 'Occurrence

Quel type de recherche pourrait être la plus fréquente sur cette table ?

Il est raisonnable de penser que les __recherches par prénom__ seront très utiles à optimiser car potentiellement les plus fréquentes.

## Comment indexer une table ?

Que pourrait-on chercher comme information tirée de cette table lorsqu'on fait une __recherche sur un prénom__ ?

Là encore, c'est subjectif, mais le plus probable est que vous souhaitiez __connaître le nombre de naissances, pour telle ou telle année__. Sur un __site web dédié aux prénoms__ par exemple.

### Indexation par prénom

Nous allons créer un index favorisant une recherche par prénom.

In [32]:
index_par_prenom = {}
for element in table_naissances:
    if element['Prénom'] in index_par_prenom:
        index_par_prenom[element['Prénom']].update({element['Année de naissance'] : element['Occurrence']})
    else:
        index_par_prenom[element['Prénom']] = {element['Année de naissance'] : element['Occurrence']}

print(index_par_prenom)

{'IRIS': {2019: 25, 2016: 6, 2009: 8, 2015: 11, 2018: 15, 2011: 7, 2017: 23, 2014: 6}, 'ROMY': {2019: 19, 2014: 7, 2018: 15, 2017: 17, 2015: 10, 2016: 15, 2013: 14, 2011: 6}, 'MARGOT': {2019: 13, 2003: 9, 2014: 17, 2013: 30, 2007: 14, 2008: 33, 2005: 15, 2018: 28, 2015: 27, 2012: 26, 2009: 24, 2002: 17, 2010: 21, 2017: 16, 2004: 12, 2011: 22, 2006: 17, 2001: 19, 2016: 15}, 'AMIRA': {2019: 11}, 'EMMY': {2019: 11, 2013: 15, 2010: 6, 2015: 6, 2014: 10, 2018: 9, 2007: 6, 2016: 8, 2012: 6, 2009: 7, 2008: 8}, 'GARANCE': {2019: 11, 2009: 12, 2013: 7, 2014: 9, 2011: 10, 2010: 7, 2007: 6, 2006: 9, 2018: 12, 2003: 7}, 'LILOU': {2019: 11, 2006: 35, 2014: 17, 2007: 43, 2017: 7, 2012: 32, 2009: 33, 2002: 11, 2018: 9, 2013: 25, 2008: 54, 2011: 29, 2010: 32, 2001: 8, 2015: 19, 2005: 35, 2016: 13, 2004: 30, 2003: 8}, 'ALICIA': {2019: 10, 2012: 12, 2008: 14, 2013: 7, 2011: 14, 2005: 13, 2002: 14, 2003: 10, 2014: 12, 2010: 18, 2007: 8, 2004: 13, 2018: 12, 2017: 14, 2006: 8, 2001: 16, 2009: 11}, 'LISON':

La recherche sur un prénom se fait maintenant de façon __très simple et très rapide__.

In [33]:
index_par_prenom['SWANN']

{2007: 7, 2013: 7, 2009: 7, 2019: 8, 2017: 6, 2012: 8, 2008: 6}

 > __Remarque :__ cette indexation est imparfaite car elle ne cumule pas les occurrences des prénoms épicènes, donnés la même année à des garçons et des filles. En voici, ci-dessous, une version plus rigoureuse.

In [34]:
index_par_prenom = {}
for element in table_naissances:
    if element['Prénom'] in index_par_prenom:
        if element['Année de naissance'] in index_par_prenom[element['Prénom']]:
            index_par_prenom[element['Prénom']][element['Année de naissance']] += element['Occurrence']
        else:
            index_par_prenom[element['Prénom']].update({element['Année de naissance'] : element['Occurrence']})
    else:
        index_par_prenom[element['Prénom']] = {element['Année de naissance'] : element['Occurrence']}

print(index_par_prenom)

{'IRIS': {2019: 25, 2016: 6, 2009: 8, 2015: 11, 2018: 15, 2011: 7, 2017: 23, 2014: 6}, 'ROMY': {2019: 19, 2014: 7, 2018: 15, 2017: 17, 2015: 10, 2016: 15, 2013: 14, 2011: 6}, 'MARGOT': {2019: 13, 2003: 9, 2014: 17, 2013: 30, 2007: 14, 2008: 33, 2005: 15, 2018: 28, 2015: 27, 2012: 26, 2009: 24, 2002: 17, 2010: 21, 2017: 16, 2004: 12, 2011: 22, 2006: 17, 2001: 19, 2016: 15}, 'AMIRA': {2019: 11}, 'EMMY': {2019: 11, 2013: 15, 2010: 6, 2015: 6, 2014: 10, 2018: 9, 2007: 6, 2016: 8, 2012: 6, 2009: 7, 2008: 8}, 'GARANCE': {2019: 11, 2009: 12, 2013: 7, 2014: 9, 2011: 10, 2010: 7, 2007: 6, 2006: 9, 2018: 12, 2003: 7}, 'LILOU': {2019: 11, 2006: 35, 2014: 17, 2007: 43, 2017: 7, 2012: 32, 2009: 33, 2002: 11, 2018: 9, 2013: 25, 2008: 54, 2011: 29, 2010: 32, 2001: 8, 2015: 19, 2005: 35, 2016: 13, 2004: 30, 2003: 8}, 'ALICIA': {2019: 10, 2012: 12, 2008: 14, 2013: 7, 2011: 14, 2005: 13, 2002: 14, 2003: 10, 2014: 12, 2010: 18, 2007: 8, 2004: 13, 2018: 12, 2017: 14, 2006: 8, 2001: 16, 2009: 11}, 'LISON':

In [35]:
index_par_prenom['SWANN']

{2007: 7, 2013: 7, 2009: 7, 2019: 8, 2017: 13, 2012: 8, 2008: 6}

Cet index nous permet également de savoir très facilement __combien de prénoms différents__ (dont l'occurrence annuelle est supérieure ou égale à 5) ont été donnés pendant cette période :

In [36]:
len(index_par_prenom)

670

Revenons aux __informations disponibles sur un prénom__...

In [37]:
index_par_prenom['DAVID']

{2019: 8,
 2014: 8,
 2018: 7,
 2003: 8,
 2006: 8,
 2012: 6,
 2004: 9,
 2017: 14,
 2013: 7,
 2010: 6,
 2009: 7,
 2002: 8,
 2015: 6,
 2001: 10}

La tentation de __trier ce résultat par date__ est forte mais...

In [38]:
index_par_prenom['DAVID'].sort()

AttributeError: 'dict' object has no attribute 'sort'

__Un dictionnaire n'étant pas ordonnée, on ne peut pas le trier !__

Qu'à cela ne tienne, il suffit de créer une liste à partir du dictionnaire.

In [39]:
liste_david = list(index_par_prenom['DAVID'].items())
liste_david.sort()

print(liste_david)

[(2001, 10), (2002, 8), (2003, 8), (2004, 9), (2006, 8), (2009, 7), (2010, 6), (2012, 6), (2013, 7), (2014, 8), (2015, 6), (2017, 14), (2018, 7), (2019, 8)]


Cet index nous permet bien un traitement aisé et efficace lors d'une recherche par prénom. Mais qu'en est-il d'une __recherche par année__ ?

### Indexation par année

Si on veut connaître les prénoms les plus donnés telle ou telle année, cet index serait particulièrement inadapté. Il faut donc créer un nouvel index dédié à ce type de recherche.

A vous de jouer...

In [40]:
index_par_annee = {}
for element in table_naissances:
    if element['Année de naissance'] in index_par_annee:
        index_par_annee[element['Année de naissance']].update({element['Prénom'] : element['Occurrence']})
    else:
        index_par_annee[element['Année de naissance']] = {element['Prénom'] : element['Occurrence']}

print(index_par_annee)

{2019: {'IRIS': 25, 'ROMY': 19, 'MARGOT': 13, 'AMIRA': 11, 'EMMY': 11, 'GARANCE': 11, 'LILOU': 11, 'ALICIA': 10, 'LISON': 10, 'COLINE': 9, 'LUCIE': 8, 'ALBA': 7, 'CÉLESTE': 7, 'FLORA': 6, 'MADELEINE': 6, 'PAULINE': 6, 'GABRIEL': 62, 'LUCAS': 45, 'GABIN': 33, 'EWEN': 13, 'LÉONARD': 12, 'KYLIAN': 11, 'JEAN': 10, 'DAVID': 8, 'TIMOTHÉ': 8, 'ISMAËL': 7, 'RAFAËL': 6, 'MILA': 39, 'SOFIA': 18, 'APOLLINE': 14, 'FAUSTINE': 13, 'ELISE': 12, 'ARIA': 11, 'LISE': 10, 'ANAËLLE': 8, 'MAYA': 7, 'RAPHAËLLE': 7, 'ISAURE': 6, 'MAUD': 6, 'YAËLLE': 6, 'ARTHUR': 67, 'LÉON': 52, 'NOAH': 33, 'ADAM': 32, 'CÔME': 27, 'MOHAMED': 27, 'CLÉMENT': 26, 'BASILE': 19, 'MILO': 17, 'AMIR': 13, 'ALEXIS': 12, 'WILLIAM': 11, 'PIERRE': 10, 'DIEGO': 9, 'ALBAN': 7, 'ANTONIN': 7, 'DANIEL': 7, 'AHMED': 6, 'AMAURY': 6, 'NOAM': 6, 'NOHAM': 6, 'HÉLOÏSE': 23, 'ANAÏS': 18, 'SALOMÉ': 17, 'YASMINE': 10, 'CÉLIA': 9, 'CLÉMENCE': 9, 'EMY': 9, 'LEÏLA': 9, 'AMÉLIA': 7, 'ELÉONORE': 7, 'NOÉ': 32, 'TIAGO': 29, 'AXEL': 25, 'VALENTIN': 20, 'NINO'

Trouver les __prénoms les plus populaires en 2008__ :

In [42]:
print(index_par_annee[2008])

{'MANON': 57, 'CHLOÉ': 41, 'JULIETTE': 35, 'ROSE': 29, 'JULIE': 25, 'MAËLYS': 25, 'CHARLOTTE': 24, 'CAPUCINE': 21, 'NOÉMIE': 18, 'LAURA': 15, 'ALICIA': 14, 'ELISA': 14, 'OCÉANE': 14, 'EMILIE': 11, 'LÉANE': 11, 'CHARLINE': 10, 'HÉLOÏSE': 9, 'LANA': 7, 'LEÏLA': 7, 'ELÉONORE': 6, 'JUDITH': 6, 'KENZA': 6, 'MARIA': 6, 'MATHIS': 57, 'CLÉMENT': 42, 'THOMAS': 35, 'NOLAN': 31, 'MARIUS': 22, 'ADRIEN': 18, 'MATHYS': 17, 'ESTEBAN': 15, 'ELIOT': 9, 'LIAM': 9, 'DYLAN': 8, 'FÉLIX': 8, 'AYMERIC': 6, 'JOSEPH': 6, 'JULIAN': 6, 'MAEL': 6, 'MELVYN': 6, 'SAMI': 6, 'ZOÉ': 44, 'MATHILDE': 26, 'LOUANE': 17, 'ELSA': 16, 'APOLLINE': 13, 'ADÈLE': 12, 'MÉLINE': 10, 'YUNA': 10, 'CLARISSE': 9, 'LOUISA': 9, 'LALY': 7, 'LOUANNE': 7, 'JOSÉPHINE': 6, 'LUCILE': 6, 'PRUNE': 6, 'ETHAN': 46, 'TITOUAN': 41, 'ANTOINE': 40, 'ARTHUR': 40, 'THÉO': 35, 'VALENTIN': 23, 'NINO': 16, 'LOUKA': 15, 'TRISTAN': 13, 'LUCIEN': 9, 'MATTHIEU': 8, 'WILLIAM': 8, 'PACÔME': 7, 'RAYAN': 7, 'SASHA': 7, 'COLIN': 6, 'DIEGO': 6, 'LINO': 6, 'CAMILLE'

Trouver les __10 prénoms__ les plus populaires en 2019 :

In [43]:
prenoms_2019 = list(index_par_annee[2019].items())

prenoms_2019.sort(key=lambda x: float(x[1]), reverse=True)

print(prenoms_2019[:10])

[('ARTHUR', 67), ('GABRIEL', 62), ('LOUISE', 62), ('ALICE', 58), ('MAËL', 56), ('LÉON', 52), ('LÉO', 52), ('RAPHAËL', 48), ('EMMA', 48), ('JADE', 46)]


### Indexation pour une recherche par sexe (Approfondissement)

Même principe que précédemment : créer un index pour anticiper une __recherche par année uniquement sur les prénoms de fille__.

In [44]:
index_par_annee_fille = {}
for element in table_naissances:
    if element['Sexe'] == 'FILLE':
        if element['Année de naissance'] in index_par_annee_fille:
            index_par_annee_fille[element['Année de naissance']].update({element['Prénom'] : element['Occurrence']})
        else:
            index_par_annee_fille[element['Année de naissance']] = {element['Prénom'] : element['Occurrence']}

print(index_par_annee_fille)

{2019: {'IRIS': 25, 'ROMY': 19, 'MARGOT': 13, 'AMIRA': 11, 'EMMY': 11, 'GARANCE': 11, 'LILOU': 11, 'ALICIA': 10, 'LISON': 10, 'COLINE': 9, 'LUCIE': 8, 'ALBA': 7, 'CÉLESTE': 7, 'FLORA': 6, 'MADELEINE': 6, 'PAULINE': 6, 'MILA': 39, 'SOFIA': 18, 'APOLLINE': 14, 'FAUSTINE': 13, 'ELISE': 12, 'ARIA': 11, 'LISE': 10, 'ANAËLLE': 8, 'MAYA': 7, 'RAPHAËLLE': 7, 'ISAURE': 6, 'MAUD': 6, 'YAËLLE': 6, 'HÉLOÏSE': 23, 'ANAÏS': 18, 'SALOMÉ': 17, 'YASMINE': 10, 'CÉLIA': 9, 'CLÉMENCE': 9, 'EMY': 9, 'LEÏLA': 9, 'AMÉLIA': 7, 'ELÉONORE': 7, 'ROSE': 44, 'ADÈLE': 43, 'AGATHE': 41, 'ANNA': 27, 'OLIVIA': 25, 'CAPUCINE': 17, 'ALMA': 12, 'LISA': 12, 'THAÏS': 12, 'ANOUK': 11, 'LOUISA': 11, 'ANAÉ': 9, 'LOUISON': 8, 'LYNA': 6, 'PIA': 6, 'LOUISE': 62, 'CHLOÉ': 43, 'LÉA': 30, 'AMBRE': 29, 'MARGAUX': 28, 'NINA': 26, 'LOLA': 22, 'VICTOIRE': 22, 'CHARLIE': 20, 'SUZANNE': 20, 'EVA': 18, 'ROXANE': 15, 'GABRIELLE': 14, 'AYA': 13, 'NORAH': 13, 'ZÉLIE': 11, 'DIANE': 10, 'ÉLISE': 9, 'MAËLYS': 9, 'MARIE': 8, 'LILY': 7, 'MAÏNA': 

Trouver les __10 prénoms de fille__ les plus populaires en 2004 :

In [46]:
prenoms_fille_2004 = list(index_par_annee_fille[2004].items())

prenoms_fille_2004.sort(key=lambda x: float(x[1]), reverse=True)

print(prenoms_fille_2004[:10])

[('MANON', 53), ('EMMA', 40), ('LÉA', 39), ('CAMILLE', 36), ('SARAH', 34), ('INÈS', 34), ('LOUISE', 32), ('CHLOÉ', 30), ('LILOU', 30), ('JULIETTE', 27)]


Créer un index pour anticiper une __recherche par sexe et par année__. C'est à dire, un index permettant de faire ce type de recherche :

`index_par_annee_et_sexe['GARCON'][2003]`

In [47]:
index_par_annee_et_sexe = {}
for element in table_naissances:
    if element['Sexe'] == 'FILLE':
        if 'FILLE' in index_par_annee_et_sexe:
            if element['Année de naissance'] in index_par_annee_et_sexe['FILLE']:
                index_par_annee_et_sexe['FILLE'][element['Année de naissance']].update({element['Prénom'] : element['Occurrence']})
            else:
                index_par_annee_et_sexe['FILLE'][element['Année de naissance']] = {element['Prénom'] : element['Occurrence']}
        else:
            index_par_annee_et_sexe['FILLE'] = {element['Année de naissance'] : {element['Prénom'] : element['Occurrence']}}
    else:
        if 'GARCON' in index_par_annee_et_sexe:
            if element['Année de naissance'] in index_par_annee_et_sexe['GARCON']:
                index_par_annee_et_sexe['GARCON'][element['Année de naissance']].update({element['Prénom'] : element['Occurrence']})
            else:
                index_par_annee_et_sexe['GARCON'][element['Année de naissance']] = {element['Prénom'] : element['Occurrence']}
        else:
            index_par_annee_et_sexe['GARCON'] = {element['Année de naissance'] : {element['Prénom'] : element['Occurrence']}}
print(index_par_annee_et_sexe)

{'FILLE': {2019: {'IRIS': 25, 'ROMY': 19, 'MARGOT': 13, 'AMIRA': 11, 'EMMY': 11, 'GARANCE': 11, 'LILOU': 11, 'ALICIA': 10, 'LISON': 10, 'COLINE': 9, 'LUCIE': 8, 'ALBA': 7, 'CÉLESTE': 7, 'FLORA': 6, 'MADELEINE': 6, 'PAULINE': 6, 'MILA': 39, 'SOFIA': 18, 'APOLLINE': 14, 'FAUSTINE': 13, 'ELISE': 12, 'ARIA': 11, 'LISE': 10, 'ANAËLLE': 8, 'MAYA': 7, 'RAPHAËLLE': 7, 'ISAURE': 6, 'MAUD': 6, 'YAËLLE': 6, 'HÉLOÏSE': 23, 'ANAÏS': 18, 'SALOMÉ': 17, 'YASMINE': 10, 'CÉLIA': 9, 'CLÉMENCE': 9, 'EMY': 9, 'LEÏLA': 9, 'AMÉLIA': 7, 'ELÉONORE': 7, 'ROSE': 44, 'ADÈLE': 43, 'AGATHE': 41, 'ANNA': 27, 'OLIVIA': 25, 'CAPUCINE': 17, 'ALMA': 12, 'LISA': 12, 'THAÏS': 12, 'ANOUK': 11, 'LOUISA': 11, 'ANAÉ': 9, 'LOUISON': 8, 'LYNA': 6, 'PIA': 6, 'LOUISE': 62, 'CHLOÉ': 43, 'LÉA': 30, 'AMBRE': 29, 'MARGAUX': 28, 'NINA': 26, 'LOLA': 22, 'VICTOIRE': 22, 'CHARLIE': 20, 'SUZANNE': 20, 'EVA': 18, 'ROXANE': 15, 'GABRIELLE': 14, 'AYA': 13, 'NORAH': 13, 'ZÉLIE': 11, 'DIANE': 10, 'ÉLISE': 9, 'MAËLYS': 9, 'MARIE': 8, 'LILY': 7,

In [48]:
index_par_annee_et_sexe['GARCON'][2003]

{'THÉO': 39,
 'ALEXIS': 23,
 'TRISTAN': 21,
 'PAUL': 19,
 'JÉRÉMY': 10,
 'TANGUY': 8,
 'LILIAN': 7,
 'ALAN': 6,
 'ELIOT': 6,
 'LENNY': 6,
 'LUCAS': 37,
 'PIERRE': 27,
 'MAXIME': 25,
 'CORENTIN': 24,
 'NOA': 13,
 'EVAN': 11,
 'KÉVIN': 11,
 'MATHIEU': 11,
 'FLORIAN': 10,
 'LORENZO': 9,
 'ARMAND': 7,
 'ELOUAN': 7,
 'KILIAN': 7,
 'ADRIEN': 6,
 'ENZO': 39,
 'ARTHUR': 29,
 'TOM': 24,
 'ALEXANDRE': 23,
 'NICOLAS': 20,
 'MAXENCE': 16,
 'DORIAN': 14,
 'JEAN': 11,
 'LOGAN': 11,
 'MAËL': 9,
 'DAVID': 8,
 'MATHYS': 8,
 'ESTEBAN': 7,
 'MARC': 7,
 'BRYAN': 6,
 'GUILLAUME': 6,
 'ROMAIN': 24,
 'VALENTIN': 19,
 'VICTOR': 19,
 'ETHAN': 12,
 'SAMUEL': 10,
 'YANIS': 9,
 'MOHAMED': 7,
 'ARNAUD': 6,
 'MORGAN': 6,
 'THOMAS': 48,
 'ANTOINE': 33,
 'BAPTISTE': 30,
 'RAPHAËL': 16,
 'KYLLIAN': 13,
 'BASTIEN': 11,
 'MATHÉO': 11,
 'MATTHIEU': 10,
 'GABRIEL': 9,
 'VINCENT': 9,
 'JONATHAN': 6,
 'SACHA': 6,
 'LÉO': 27,
 'MARTIN': 22,
 'KYLIAN': 14,
 'EWEN': 8,
 'GIOVANNI': 8,
 'YANN': 8,
 'GAËTAN': 7,
 'JASON': 6,
 'Q

Trouver les __10 prénoms de garçon les plus populaires en 2003__ :

In [49]:
prenoms_garcon_2003 = list(index_par_annee_et_sexe['GARCON'][2003].items())

prenoms_garcon_2003.sort(key=lambda x: float(x[1]), reverse=True)

print(prenoms_garcon_2003[:10])

[('THOMAS', 48), ('NATHAN', 40), ('THÉO', 39), ('ENZO', 39), ('MATHIS', 38), ('LUCAS', 37), ('CLÉMENT', 37), ('HUGO', 35), ('QUENTIN', 34), ('ANTOINE', 33)]


---
[![Licence CC BY NC SA](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png "licence Creative Commons CC BY-NC-SA")](http://creativecommons.org/licenses/by-nc-sa/3.0/fr/)
<p style="text-align: center;">Auteur : David Landry, Lycée Clemenceau - Nantes</p>